In [ ]:
"""
============================================================
SIMPLE ANALYTICS FOR ALPACA OPTIONS
------------------------------------------------------------
Author : Shawn Nordstrom (Yale University)
Purpose: Sandbox for trade analytics.
Notes  :
 - Trades are grouped by open/close
 - Built explicitly for single-position long options trading.
 - Look out for what is calculated as net vs gross!
============================================================
Version 0.0.2 | Date: 2025-11-06
"""

In [ ]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import QueryOrderStatus, PositionIntent
import pandas as pd
import numpy as np


#                       CONFIGURATION
# =========================================================
API_KEY = ""
SECRET_KEY = ""
start_capital = 80000
cutoff = pd.Timestamp("2025-11-09", tz="UTC")
# =========================================================

trading_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=True)
orders = trading_client.get_orders(
    GetOrdersRequest(
        status=QueryOrderStatus.ALL  # OPEN, CLOSED
    )
)

In [ ]:
# Load data
data = [dict(order) for order in orders]
df = pd.DataFrame(data)
df = df.sort_values(by='filled_at')
rows = []
for i in range(len(df) - 1):
    current = df.iloc[i]
    next_row = df.iloc[i + 1]
    
    if current['position_intent'] in (PositionIntent.BUY_TO_OPEN, PositionIntent.SELL_TO_OPEN):
        if current['filled_qty'] != next_row['filled_qty']:
            print("Error! Mismatched fill qty")
            continue
        rows.append({
            'open_create_time' : current['created_at'],
            'open_filled_time' : current['filled_at'],
            'close_filled_time' : next_row['filled_at'],
            'open_price' : current['filled_avg_price'],
            'close_price' : next_row['filled_avg_price'],
            'qty' : current['filled_qty'],
            'qty_2' : next_row['filled_qty']
        })
trades = pd.DataFrame(rows)
trades['open_price']  = pd.to_numeric(trades['open_price'], errors='coerce')
trades['close_price'] = pd.to_numeric(trades['close_price'], errors='coerce')
trades['qty'] = pd.to_numeric(trades['qty'], errors='coerce')
trades["pnl_pct"] = (trades['close_price'] - trades['open_price'])/trades['open_price'] * 100
trades["date"] = trades["close_filled_time"].dt.date
trades = trades[trades["open_create_time"] <= cutoff]

In [ ]:
trades

In [ ]:
# Trade Statistics
wins = (trades["pnl_pct"] > 0).sum()
losses = (trades["pnl_pct"] < 0).sum()
flat = (trades["pnl_pct"] == 0).sum()

avg = trades["pnl_pct"].mean()
avg_win = trades.loc[trades["pnl_pct"] > 0, "pnl_pct"].mean()
avg_loss = trades.loc[trades["pnl_pct"] < 0, "pnl_pct"].mean()


gross_profit = trades.loc[trades["pnl_pct"] > 0, "pnl_pct"].sum()
gross_loss = -trades.loc[trades["pnl_pct"] < 0, "pnl_pct"].sum()
profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

print(f"Total number of trades: {len(trades)}\nNumber of wins: {wins}\nNumber of losses: {losses}\nFlat: {flat}\n\nWin Percent: {wins/len(trades)*100}%\n\nAverage trade: {avg:.2f}%\nAverage win: {avg_win:.2f}%\nAverage loss: {avg_loss:.2f}%\n\nProfit Factor: {profit_factor:.2f}")

In [ ]:
# Gross capital (ignoring fees)
gross_capital = start_capital
gross_capital_curve = []
pnl_list = []
for i, row in trades.iterrows():
    pnl = (row['close_price'] - row['open_price']) * row['qty'] * 100
    gross_capital += pnl
    pnl_list.append(pnl)
    gross_capital_curve.append(gross_capital)

trades['pnl'] = pnl_list
trades['gross_capital'] = gross_capital_curve
print(f"Starting Capital: {start_capital}\nEnding Gross Capital: {gross_capital_curve[-1]:.2f}")

In [ ]:
# Gross P/L
pnl_g = trades['gross_capital'].iloc[-1] - start_capital
pnl_g_pct = pnl_g / start_capital * 100

print(f"Gross P/L: {pnl_g:.2f}\nGross P/L Percent: {pnl_g_pct:.2f}%")

In [ ]:
# Max Drawdown (of gross capital)
max_dd = 0
peak = start_capital
for i, row in trades.iterrows():
    if row['gross_capital'] >= peak:
        peak = row['gross_capital']
    current_drawdown = 0
    if peak != row['gross_capital']:
        current_drawdown = (peak - row['gross_capital'])/peak
    if current_drawdown > max_dd:
        max_dd = current_drawdown
max_dd *= 100
print(f"Max Drawdown: {max_dd:.2f}%")

In [ ]:
# Trade breakdown (use Data Wrangler)
trades

In [ ]:
# Per day statistics (requires running gross capital cell)
daily_end = trades.groupby("date")["gross_capital"].last()
initial_capital = 80000
daily_start = daily_end.shift(1)
daily_start.iloc[0] = initial_capital
daily_stats = (
    pd.DataFrame({
        "date": daily_end.index,
        "start_gross_capital": daily_start.values,
        "end_gross_capital": daily_end.values,
    })
    .reset_index(drop=True)
)
daily_stats["daily_ret_pct"] = (daily_stats["end_gross_capital"] / daily_stats["start_gross_capital"] - 1.0) * 100
daily_stats

In [ ]:
# T-stat
import numpy as np
r = trades["pnl_pct"]
n = r.size
mu = r.mean()
s = r.std(ddof=1)
t_trade = mu / (s / np.sqrt(n)) if n > 1 and s > 0 else np.nan
print(f"n_trades: {n}\nmean_trade: {mu}\nstd_trade: {s}\nt_stat_trade: {t_trade}")